In [1]:
%run functions.ipynb

np.set_printoptions(precision=3)
np.random.seed(42)

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train, X_test, y_train, y_test = load_train_and_test_parts()
print("Train and test sizes: {} {}".format(X_train.shape, X_test.shape))
print("(1, 0) labels count in train test: {} {}".format((np.count_nonzero(y_train==1), np.count_nonzero(y_train==0)), 
                                                        (np.count_nonzero(y_test==1), np.count_nonzero(y_test==0))))

Train and test sizes: (726, 12179) (243, 12179)
(1, 0) labels count in train test: (289, 437) (104, 139)


In [3]:
n_iter = 3
cv = 4
n_jobs = 3
scoring = 'roc_auc'

In [4]:
# Parameters
scoring = "recall"
cv = 4
n_iter = 30
n_jobs = 4


In [5]:
nsc_params = {'shrink_threshold' : np.linspace(0, 5, 50)}
clf_nsc = RandomizedSearchCV(NearestCentroidWrapper(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=nsc_params)  
clf_nsc = fit_clf_print_scores(clf_nsc, X_train, y_train, X_test, y_test)

NearestCentroidWrapper(metric='euclidean',
            shrink_threshold=4.8979591836734695)


,ROC AUC,Precision,Recall,F1
train,0.686,0.603,0.657,0.629
test,0.624,0.565,0.587,0.575


In [6]:
clf_lda = fit_clf_print_scores(LinearDiscriminantAnalysis(shrinkage='auto', solver='lsqr'), X_train, y_train, X_test, y_test)

,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.796,0.753,0.615,0.677


In [7]:
logistic_params = {'C' : np.linspace(0.01, 0.6, 50), 'penalty' : ['l1'], 'solver' : ['liblinear']}
tree_params = {'n_estimators' : [1000, 2500, 5000], 'max_depth': [4, 6, 8], 'min_samples_leaf' : [20, 30]}

In [8]:
logit_cv = RandomizedSearchCV(LogisticRegression(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=logistic_params) 
tree_cv = RandomizedSearchCV(RandomForestClassifier(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=tree_params) 

clf_logit = fit_clf_print_scores(logit_cv, X_train, y_train, X_test, y_test)
clf_forest = fit_clf_print_scores(tree_cv, X_train, y_train, X_test, y_test)

LogisticRegression(C=0.4675510204081632, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


,ROC AUC,Precision,Recall,F1
train,1.000,0.996,0.983,0.990
test,0.809,0.733,0.712,0.722


/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 18 is smaller than n_iter=30. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


,ROC AUC,Precision,Recall,F1
train,0.964,0.940,0.765,0.844
test,0.777,0.739,0.490,0.590


In [9]:
elastic_params = {'l1_ratio' : np.linspace(0.001, 1, 50), 'alpha' : np.linspace(0.0001, 0.5, 100)}
elastic_cv = RandomizedSearchCV(SGDClassifier(loss='log', penalty='elasticnet', max_iter=5000, tol=1e-3), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=elastic_params)
clf = fit_clf_print_scores(elastic_cv, X_train, y_train, X_test, y_test)

SGDClassifier(alpha=0.03544646464646465, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.041775510204081635, learning_rate='optimal', loss='log',
       max_iter=5000, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)


,ROC AUC,Precision,Recall,F1
train,0.843,0.732,0.754,0.743
test,0.797,0.700,0.673,0.686


In [10]:
svm_params = {'C' : np.linspace(0.01, 0.6, 50), 'kernel' : ['poly', 'rbf', 'sigmoid']}
svm_cv = RandomizedSearchCV(SVC(probability=True, gamma='scale'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=svm_params)

clf_svm = fit_clf_print_scores(svm_cv, X_train, y_train, X_test, y_test)

SVC(C=0.39530612244897956, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.800,0.719,0.663,0.690


In [11]:
from sklearn.svm import LinearSVC

svm_params = {'C' : np.linspace(0.01, 0.6, 50)}
svm_cv = RandomizedSearchCV(LinearSVC(penalty='l1', dual=False, max_iter=10000), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=svm_params)

clf_svm = fit_clf_print_scores(svm_cv, X_train, y_train, X_test, y_test)

LinearSVC(C=0.4555102040816326, class_weight=None, dual=False,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=10000, multi_class='ovr', penalty='l1', random_state=None,
     tol=0.0001, verbose=0)


/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.782,0.690,0.663,0.676


In [12]:
rlr_params = {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000], 'n_variables' : [100, 250, 500, 1000]}
rlr_cv = RandomizedSearchCV(RandomLogisticsRegressions(penalty='l1'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=rlr_params)

clf_rlr = fit_clf_print_scores(rlr_cv, X_train, y_train, X_test, y_test)

/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


RandomLogisticsRegressions(C=4.6, n_estimators=1000, n_variables=250,
              penalty='l1', solver='liblinear', tol=0.0001)


,ROC AUC,Precision,Recall,F1
train,0.989,0.978,0.910,0.943
test,0.817,0.716,0.654,0.683
